In [1]:
import numpy as np
import struct
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline
from tqdm import tqdm

## Read dynamic list-mode CASToR datafile

In [16]:
datafile='data/poumon_pat6_list-mode_0_tof_ref0_compr_1_100.cdf'

In [17]:
# Structure of datafile for 32 bit list-mode ( including TOF )
struct.calcsize('IfffffII')


# Structure of compressed datafile for 24 bit list-mode ( including TOF )
# uint32_t : time tag
# float: Normalization + Attenuation
# float: Scatter + randoms
# float: TOF arrival time diff
# uint32_t: CrystalID1
# uint32_t: CrystalID2
struct.calcsize('IfffII')

24

In [18]:
# Get last time tag
with open(datafile, mode='rb') as file: # b is important -> binary
    file.seek(os.path.getsize(datafile)-24)
    # Read last phrase
    datafilePack = file.read(24)
    # read its time tag , as the last time tag
    lastTimeTag = int(struct.unpack("I",datafilePack[:4])[0])

In [19]:
lastTimeTag

3599999

## Define Framing for the study ( on study refference time == injection time )

In [20]:
frm="0,10,20,30,40,50,60,70,80,90,100, \
110,120,140,160,180,200,220,240,260,280,300,320,340,   \
360,420,480,540,600,720,840,960,1080,1200,1500,1800,2100,2400,2700,3000,3300:300"

In [21]:
frames_phrases=frm.split(",")
FrameStart=np.zeros([len(frames_phrases)],dtype='int')
FrameDuration=np.zeros([len(frames_phrases)],dtype='int')

fr=0
for phrase in frames_phrases:
    #print(phrase)
    if (phrase.find(":") != -1) :
        FrameStart[fr]=float(phrase.split(":")[0])
        FrameDuration[fr]=float(phrase.split(":")[1])
    else:
        FrameStart[fr]=float(phrase)
    fr+=1

# Loop over farmes duration and replace zeros with time until next frame
for idx,val in enumerate(FrameDuration):
    if val==0:
        FrameDuration[idx]=FrameStart[idx+1]-FrameStart[idx]

## Read respiratory triggers time tags

In [22]:
resp_triggers=pd.read_csv('data/LIST0000.resp')
resp_triggers.columns = ['trigger_time_tag']

In [23]:
# The data were unlisted from 60 seconds onwards ( and refference is 60sec)
# We need to apply the same offset to the respiratory gates
resp_triggers = (resp_triggers-60000)
# Then remove triggers taking place before injection time (time=0)
resp_triggers = resp_triggers[resp_triggers['trigger_time_tag'] > 0]

In [24]:
resp_triggers.diff().mean()

trigger_time_tag    3419.779258
dtype: float64

In [25]:
resp_triggers = np.array(resp_triggers.trigger_time_tag.tolist())
resp_triggers_diff = np.diff(resp_triggers)

In [26]:
# Statistics of resp tags
print("Number of respiratory tags: %d"%resp_triggers.shape[0])
print("Mean respiratory rate: %f ms"%np.diff(resp_triggers).mean())
accepted_count=((np.diff(resp_triggers)<4000*1.66) & (np.diff(resp_triggers)>4000*0.33)).sum()
print("Number of cycles to include: %d "%accepted_count)
print("Rejection fraction: {0:.4f}%".format((1-accepted_count/resp_triggers.shape[0])*100))

Number of respiratory tags: 1052
Mean respiratory rate: 3419.779258 ms
Number of cycles to include: 952 
Rejection fraction: 9.5057%


In [27]:
# Check if last respiratory trigger takes place before end of file
print (resp_triggers[-1]<lastTimeTag)

True


## Create time-tag info array for gates phase, gate index and rejected triger phases

### In this step we create an information matrix for each time-tag within the list mode file.
In each time tag we have the following information:
#0: time-tag
#1: respiratory cycle index
#2: Frame index
#3: Accepted or rejected  based on respiratory cycle duration ( within 30% of 3000ms ) 
#4: Use for Q.Static ?  (Yes=1 or No=0)
#5: Q.Freeze (Gate) index number

In [28]:
InfoMatrix = np.zeros([lastTimeTag+1,6],dtype=int)

In [29]:
InfoMatrix[:,0] = np.arange(lastTimeTag+1)

In [30]:
# Set respiratory cycle index and check respiratory cycle duration
current_resp_cycle=0
current_frame=0
# Reject all tags before the first respiratory trigger and after the last 
for tag in range(resp_triggers[0]):
    InfoMatrix[tag,3]=0

for tag in range(resp_triggers[-1],lastTimeTag):
    InfoMatrix[tag,3]=0

for tag in tqdm(range(resp_triggers[0],resp_triggers[-1])):
    # Check if we need to update frame index
    if tag>=((FrameStart[current_frame]+FrameDuration[current_frame])*1000):
        current_frame+=1;
    InfoMatrix[tag,2]=current_frame
    
    # it tag is greater or equal to the following resp trigger, move one cycle up
    if tag>=resp_triggers[current_resp_cycle+1]:
        current_resp_cycle+=1;
    # Set the current_resp_cycle index
    InfoMatrix[tag,1]=current_resp_cycle
    
    # Check duration of cycle to reject or keep
    if (4000*0.33<resp_triggers_diff[current_resp_cycle]<4000*1.66):
        InfoMatrix[tag,3]=1
    
    # Trigger is already placed 30% forward from the peak ( by GE )
    # we keep the first 50% of the data in the resp cycle for Q.Static data
    if (tag<resp_triggers[current_resp_cycle]+resp_triggers_diff[current_resp_cycle]/2):
        InfoMatrix[tag,4]=1
      
    # Trigger is placed 30% forward from the peak ( by GE )
    # we splitt the data in the resp cycle in 5 parts/gates for Q.Static data    
    for gate in range(1,6):
        gate_start=resp_triggers[current_resp_cycle]+(resp_triggers_diff[current_resp_cycle]/5.)*(gate-1)
        gate_end=resp_triggers[current_resp_cycle]+(resp_triggers_diff[current_resp_cycle]/5.)*(gate)
        if (gate_start<=tag<gate_end):
            InfoMatrix[tag,5]=gate


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3594188/3594188 [04:18<00:00, 13900.79it/s]


In [31]:
np.savetxt('data/InfoMatrix.txt',InfoMatrix.astype(int),fmt='%i', delimiter=",")

In [44]:
InfoMatrix = np.loadtxt('data/InfoMatrix.txt', delimiter=",")

In [ ]:
#InfoMatrix[1495:1555]
InfoMatrix[:2614]
InfoMatrix[360000:360000+60000,3].sum()

## Process the list-mode file and create Q.Static Q.Freeze list files (with frames)

In [ ]:
# Q.Static
pbar = tqdm(total=int(os.path.getsize(datafile)/24))   
    
with open(datafile, mode='rb') as file:
    # Create Q.Static file
    QStaticFile = open(os.path.basename(datafile).split('.')[0]+'_QStatic.cdf', "wb")
    
    # Read first datafilePack
    datafilePack = file.read(24)
    count=1
    QStaticCount=0
    # Loop over all data untill end of file
    while datafilePack:
        # get time tag
        timetag = int(struct.unpack("I",datafilePack[:4])[0])
        # Check if we will use this time tag
        if InfoMatrix[timetag,2]==1:
            count+=1
            # Check if we will use this time tag for Q.Static
            if InfoMatrix[timetag,3]==1:
                QStaticCount+=1
                QStaticFile.write(datafilePack)

        # Read next datafilePack
        datafilePack = file.read(24)
        #print progress
        pbar.update(int(count))

    # close output files
    QStaticFile.close()
    # close progress bar 
    pbar.close()

In [ ]:
print("Number of events in Q.Static CASToR Datafile: %d"%QStaticCount)
print("Number of total events (accepted) from original CASToR Datafile: %d"%count)

In [ ]:
InfoMatrix[2614]

In [ ]:
with open(datafile, mode='rb') as file:
    datafilePack = file.read(24)
    while datafilePack:
        timetag = int(struct.unpack("I",datafilePack[:4])[0])
        if (timetag==2615):
            print("found")
            break
        datafilePack = file.read(24)

In [ ]:
datafilePack

In [ ]:
struct.unpack('IfffII', datafilePack)

In [ ]:
# Q.Freeze
pbar = tqdm(total=int(os.path.getsize(datafile)/24))   
    
with open(datafile, mode='rb') as file:
    # Create Q.Static file
    
    QFreezeFile_gate1 = open(os.path.basename(datafile).split('.')[0]+'_QFreeze_Gate1.cdf', "wb")
    QFreezeFile_gate2 = open(os.path.basename(datafile).split('.')[0]+'_QFreeze_Gate2.cdf', "wb")
    QFreezeFile_gate3 = open(os.path.basename(datafile).split('.')[0]+'_QFreeze_Gate3.cdf', "wb")
    QFreezeFile_gate4 = open(os.path.basename(datafile).split('.')[0]+'_QFreeze_Gate4.cdf', "wb")
    QFreezeFile_gate5 = open(os.path.basename(datafile).split('.')[0]+'_QFreeze_Gate5.cdf', "wb")
    QFreezeFiles=[QFreezeFile_gate1,QFreezeFile_gate2,QFreezeFile_gate3,QFreezeFile_gate4,QFreezeFile_gate5]
    
    # Read first datafilePack
    datafilePack = file.read(24)
    count=1
    GateCount=np.zeros([5])
    # Loop over all data untill end of file
    while datafilePack:
        # get time tag
        timetag = int(struct.unpack("I",datafilePack[:4])[0])
        # Check if we will use this time tag
        if InfoMatrix[timetag,2]==1:
            count+=1
            # Check to which gate this event bellongs to
            gate =  int(InfoMatrix[timetag,4])
            QFreezeFiles[gate-1].write(datafilePack)
            GateCount[gate-1]+=1


        # Read next datafilePack
        datafilePack = file.read(24)
        #print progress
        pbar.update(int(count))

    # close output files
    QStaticFile.close()
    # close progress bar 
    pbar.close()

In [ ]:
QFreezeFiles[1-1]

## Calculate quantification factors

In [30]:
head_curve=pd.read_csv('data/prompts_headcurve.hc',sep=',',header=None)
head_curve.columns=['time','prompts']
prompt_array = np.array(head_curve.prompts)

In [33]:
prompt_array = np.array(head_curve.prompts)

In [40]:
prompt_array.shape

(3599999,)

In [47]:
prompt_array[tag]

IndexError: index 3599999 is out of bounds for axis 0 with size 3599999

In [53]:
QStatic_counts_per_frame=np.zeros([np.size(FrameStart)],dtype='int')
QStatic_mseconds_per_frame=np.zeros([np.size(FrameStart)],dtype='int')

for frm in range(np.size(FrameStart)):
    for tag in range(FrameStart[frm]*1000,FrameStart[frm]*1000+FrameDuration[frm]*1000):
        if ((InfoMatrix[tag,3]==1)and(InfoMatrix[tag,2]==1)):
            QStatic_counts_per_frame[frm]+=prompt_array[tag]
            QStatic_mseconds_per_frame[frm]+=1

In [54]:
for frm in range(np.size(FrameStart)):
    print(str(QStatic_counts_per_frame[frm])+", 0")

1143217, 0
14401275, 0
12604682, 0
9752994, 0
9674381, 0
10211739, 0
8830682, 0
9813369, 0
5479591, 0
8652089, 0
9549498, 0
9884698, 0
17564165, 0
16746187, 0
17749543, 0
11930123, 0
12077855, 0
16633524, 0
15527830, 0
17696646, 0
12271621, 0
15865530, 0
8792135, 0
12064577, 0
45112196, 0
47135635, 0
46051818, 0
39658385, 0
85219663, 0
85218879, 0
48271429, 0
62289055, 0
35531225, 0
74543813, 0
122054196, 0
126281448, 0
129060807, 0
119510445, 0
116422951, 0
112176136, 0
105580884, 0


In [56]:
for frm in range(np.size(FrameStart)):
    print(str(QStatic_mseconds_per_frame[frm]/1000)+", 0")

3.24, 0
3.53, 0
4.85, 0
5.1, 0
4.87, 0
5.4, 0
4.707, 0
5.23, 0
2.92, 0
4.631, 0
5.142, 0
5.421, 0
9.928, 0
9.602, 0
10.32, 0
7.0, 0
7.17, 0
9.98, 0
9.33, 0
10.73, 0
7.52, 0
9.781, 0
5.429, 0
7.48, 0
27.942, 0
29.998, 0
29.974, 0
26.236, 0
57.75, 0
59.8, 0
34.95, 0
47.13, 0
27.42, 0
61.87, 0
108.73, 0
117.726, 0
130.984, 0
130.95, 0
134.331, 0
136.92, 0
136.45, 0
